In [2]:
import xlrd
import os 
import sqlite3
import csv

In [3]:
db_path = './datawarehouse.db'
con=sqlite3.connect(db_path)
cur = con.cursor()

query = '''SELECT * from listings'''

cur.execute(query)
r = cur.fetchone()

print(r)

con.close()

(22229408, 'https://www.airbnb.com/rooms/22229408', 20230319041206, '2023-03-19', "Explorer's Paradise: Near Train, Bus & Bike Share", 'Welcome to your perfect urban escape in the Nation’s Capital! Only .4 miles (10 min walk) from the Metro train & Capital Bikeshare, and a 10 min Metro ride to major DC attractions, this oasis has all you need to enjoy a weekend or extended stay. Perfect space for singles and couples.<br /><br /><b>Guest access</b><br />This remodeled home has 2 stories of space to enjoy when staying here for your visit in the DC area. Guests have own private room that includes: queen bed with high quality bed linens/blankets, ceiling fan, private closet with hamper and ample clothes hangers, luggage rack, and full-length mirror. Adjacent to the guest room is a completely remodeled bathroom, just for use by guests, furnished with plush towels and hair dryer. Bedroom and bathroom are on the upper story of the home. The other shared areas that are accessible to our guests

# Backup

In [10]:
# Full path to xls, including filename
xls_path = './primary/airbnb_data_dict.xls'
airbnb_file = xlrd.open_workbook_xls(xls_path)
airbnb_var_sheets = airbnb_file.sheets()

variable_dict = {}
print('Saved Data Dictonaries:')
for s in airbnb_var_sheets:
    s_name = s.name
    var_stored = True
    
    if s_name.rfind('listings.csv detail v4') != -1:
        variable_dict['listings'] = s
        
    elif s_name.rfind('reviews.csv v1') != -1:
        variable_dict['reviews'] = s
        
    elif s_name.rfind('calendar.csv v2') != -1:
        variable_dict['calendar'] = s
        
    else:
        var_stored = False
        
    if var_stored:
        print(f'\t{s_name}')

Saved Data Dictonaries:
	listings.csv detail v4.3
	listings.csv detail v4
	reviews.csv v1
	calendar.csv v2


In [49]:
# Automatically create the listings table
cur_sheet = variable_dict['listings']
field_vals = cur_sheet.col_values(0)[8:82]
type_vals = cur_sheet.col_values(1)[8:82]

out = f'''-- Table: listings
DROP TABLE IF EXISTS listings;
CREATE TABLE listings(
    id int PRIMARY KEY,\n'''

for val in range (1, len(field_vals)-1):
    var_type = type_vals[val]
    if var_type == 'boolean [t=true; f=false]':
        var_type = 'boolean'
    if var_type == '':
        var_type = 'blob'
    out+=f'''    {field_vals[val]} {var_type},\n'''
    
out+=f'''    {field_vals[len(field_vals)-1]} {var_type}
);\n\n'''

In [50]:
print(len(field_vals))

74


In [45]:
# Automatically create the reviews table
cur_sheet = variable_dict['reviews']
field_vals = cur_sheet.col_values(0)[8:14]
type_vals = cur_sheet.col_values(1)[8:14]

out += f'''-- Table: reviews
DROP TABLE IF EXISTS reviews;
CREATE TABLE reviews(
    id int PRIMARY KEY,
    listing_id int,\n'''

for val in range (1, len(field_vals)-1):
    var_type = type_vals[val]
    if var_type == 'boolean [t=true; f=false]':
        var_type = 'boolean'
    if var_type == '':
        var_type = 'blob'
    out+=f'''    {field_vals[val]} {var_type},\n'''
    
out+=f'''    {field_vals[len(field_vals)-1]} {var_type}
);\n\n'''

In [46]:
# Automatically create the reviews table
cur_sheet = variable_dict['calendar']
field_vals = cur_sheet.col_values(0)[8:15]
type_vals = cur_sheet.col_values(1)[8:15]

out += f'''-- Table: calendar
DROP TABLE IF EXISTS calendar;
CREATE TABLE calendar(
    id int PRIMARY KEY,
    listing_id int,\n'''

for val in range (1, len(field_vals)-1):
    var_type = type_vals[val]
    if var_type == 'boolean [t=true; f=false]':
        var_type = 'boolean'
    if var_type == '':
        var_type = 'blob'
    if field_vals[val]!='id':
        out+=f'''    {field_vals[val]} {var_type},\n'''
    
out+=f'''    {field_vals[len(field_vals)-1]} {var_type}
);\n\n'''

In [47]:
text_file = open("datawarehouse.sql", "w")
n = text_file.write(out)
text_file.close()

In [38]:
field_vals

['listing_id',
 'date',
 'available',
 'price',
 'adjusted_price',
 'minimum_nights',
 'maximum_nights']

In [ ]:


out = f'''-- Table: listings
DROP TABLE IF EXISTS listings;
CREATE TABLE listings(
    id int PRIMARY KEY,\n'''

